# Group Project for Big Data Course

#TASK 1

In [3]:
#importing the dataset
import pandas as pd
ratingsURL = 'https://csc8101storageblob.blob.core.windows.net/datablobcsc8101/ratings.csv'
ratings = spark.createDataFrame(pd.read_csv(ratingsURL))

In [4]:
# loading necessary library for the project
import pandas as pd
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql import Row
from operator import add
from collections import Counter
from pyspark.sql.functions import col, sum
import os,sys
from functools import reduce
from pyspark.sql.functions import col, lit, when
from graphframes import*
import networkx as nx
import numpy as np

## Data Exploration

In [6]:
# total number of data
ratings.count()

Out[6]: 20000263

###Graphical and Numerical Summaries

#### Ratings Histogram

In [9]:
display(ratings)

userId,movieId,rating,timestamp
1,2,3.5,1112486027
1,29,3.5,1112484676
1,32,3.5,1112484819
1,47,3.5,1112484727
1,50,3.5,1112484580
1,112,3.5,1094785740
1,151,4.0,1094785734
1,223,4.0,1112485573
1,253,4.0,1112484940
1,260,4.0,1112484826


#### Average number of ratings per users

In [11]:
# To calculate the average number of ratings per user, the ratings dataframe is grouped by the userId and the entries per user counted. 
# In the next command the created dataframe with number of ratings per user is then used to calculate the average. 
from pyspark.sql.functions import avg
ratingsPerUser = ratings.select("userId", "rating").groupBy("userId").count()
ratingsPerUser.show(5)

+------+-----+
userId|count|
+------+-----+
 11276| 92|
 11434| 1696|
 11567| 107|
 11619| 83|
 11625| 96|
+------+-----+
only showing top 5 rows

In [12]:
ratingsPerUser.agg({"count": 'avg'}).show()

+-----------------+
 avg(count)|
+-----------------+
144.4135299257002|
+-----------------+

#### Average number of ratings per movie

In [14]:
# Calculating the average number of ratings per movie is similarly done to the average number of ratings per user, only that the ratings dataframe is this time grouped by the movieId. 
ratingsPerMovie = ratings.select("movieId", "rating").groupBy("movieId").count()

In [15]:
ratingsPerMovie.agg({"count": 'avg'}).show()

+-----------------+
 avg(count)|
+-----------------+
747.8411232425965|
+-----------------+

#### Histogram of the distribution of movie ratings per user

In [17]:
# Creating the two dataframes 'ratingsPerUser' and 'ratingsPerMovie' simplify the creation of a histogram showing the distribution of the ratings. 
display(ratingsPerUser)

userId,count
11276,92
11434,1696
11567,107
11619,83
11625,96
11745,71
11938,81
11945,36
12044,1385
12568,29


#### Histogram showing the distribution of movie ratings per user

In [19]:
display(ratingsPerMovie)

movieId,count
29,8520
2529,12702
474,18836
45726,1017
60756,1467
71530,1272
3764,987
1950,3449
3091,1333
106100,1020


# TASK 2

#### Data Cleaning

In [22]:
# timestamp column is dropped as it will not be useful for the rest of analysis.
ratings = ratings.drop('timestamp')

#### Test Train split

In [24]:
# The data will be split into a trainning for the fit and a testing set for the evaulation later. A 50% 50% train and test split is used here as with the rest of the models used for this task.
(train, test) = ratings.randomSplit([0.50, 0.50], seed = 1234)

#### ALS Construction

In [26]:
# The ALS is constructed by using the userIds to represent users, the movieIds to represent the items and the ratings column to represent the rating to fill.  Since the rating is not negative it, the non negative is True and preferences are not implicit. Also, a cold start strategy of drop is used to resolve the cold start problem (users with no reviews).

als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",
          nonnegative = True, implicitPrefs = False,
          coldStartStrategy = 'drop')

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

#### Tuning Hyperparameters using Cross Validation

In [28]:
# By using k fold cross validation, we can compare ALS models using different hyper parameters to pick the ideal hyperparameters. For ALS, the parameters we will tune will be the rank & regParam which refers to the number of features to discover throughout the run.
# Grid
paramGrid = ParamGridBuilder() \
           .addGrid(als.rank, [10, 5, 1]) \
           .addGrid(als.regParam, [0.001,0.01,0.1])\
           .build()

#### Cross Validation and Best Model

In [30]:
cv = CrossValidator(
  estimator= als, estimatorParamMaps=paramGrid,
  evaluator=evaluator, numFolds=3)

In [31]:
# Running cross validation and choosing best model
cvmodel = cv.fit(train)
bestModel = cvmodel.bestModel

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [32]:
# Rank and regParam of model
print("rank: ", bestModel.rank, "\nregParam: ", bestModel._java_obj.parent().getRegParam())

rank: 10 
regParam: 0.1

#### Best Model Results

In [34]:
predictions = bestModel.transform(test)
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.8210794641715616

In [35]:
display(predictions)

userId,movieId,rating,prediction
96393,148,3.0,2.6037807
136222,148,2.0,2.651352
137949,148,4.0,3.1345928
20132,148,3.0,2.5903451
44979,148,3.0,2.7873907
81218,148,1.0,2.5750425
1259,148,5.0,3.3072019
41389,148,3.0,3.3154135
60081,148,2.0,2.8515894
46146,148,2.0,2.002709


The model seems to quite well here considering the sparsity of the matrix, at least according to the root mean square error. However, perhaps the algorithm can be improved by feeding it's communities of users instead of the whole users datasets, where users of said communities are likely to have similar prefereneces and hence predicting ratings would be easier. This will be done using Girwan Newman in the remaining tasks.

# TASK 3

In [38]:
# Taking 27000 users from the dataset
sampleUsers1 = ratings.select("userId").distinct().sample(fraction=0.2)
list = sampleUsers1.take(27000)
sampleUsers = spark.createDataFrame(list,['userId'])
sampleUsers.count()

Out[23]: 27000

In [39]:
# creating ratings_small file for parquet
ratings_small = ratings[ratings["userId"].isin(sampleUsers.toPandas()["userId"].tolist())]
ratings_small.count()

Out[24]: 3876959

In [40]:
# saving ratings_small as parquet file. This won't work if file already exists.
 ratings_small.write.parquet("ratings-small.parquet")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o35411.parquet.
: org.apache.spark.sql.AnalysisException: path dbfs:/ratings-small.parquet already exists.;
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:148)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:126)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:146)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:187)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:183)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:115)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:115)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:240)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:170)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:306)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:292)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:235)
	at org.apache.spark.sql.DataFrameWriter.parquet(DataFrameWriter.scala:600)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 AnalysisException Traceback (most recent call last)
 <command-1004304869416591> in <module> 
 1 # saving ratings_small as parquet file. This won't work if file already exists. 
 ----> 2 ratings_small . write . parquet ( "ratings-small.parquet" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, path, mode, partitionBy, compression) 
 842 self . partitionBy ( partitionBy ) 
 843 self . _set_opts ( compression

# TASK 4

In [42]:
# dropping 'timestamp' & 'rating' as they are not necessary for analysis for rest of the project.
ratings_small = ratings_small.drop('timestamp')
ratings_small = ratings_small.drop('rating')

#### Dataframes

In [44]:
# Two dataframes will be used to help divide the task up, each representing one of the user ids in the edge
ratings_small = spark.read.parquet("ratings-small.parquet")
df1 = ratings_small.select("movieId", "userId")
df2 = ratings_small.select("movieId", "userId").withColumnRenamed("movieId", "movieId2").withColumnRenamed("userId", "userId2")
UserMap = df1.join(df2, df1.movieId == df2.movieId2)

In [45]:
UserMap = UserMap.select("userId", "userId2")
UserMap.show(5)

+------+-------+
userId|userId2|
+------+-------+
 69144| 34375|
 69144| 34364|
 69144| 34357|
 69144| 34350|
 69144| 34337|
+------+-------+
only showing top 5 rows

In [46]:
UserMap.count()

Out[29]: 10177306061

#### Edges

In [48]:
# finding duplicate edges
edgesWithDuplicates = UserMap.groupBy(UserMap.columns).count().filter(UserMap.userId != UserMap.userId2)
edgesWithDuplicates.orderBy("count", ascending=False).show(5)

In [49]:
# total number of duplicate edges
edgesWithDuplicates.count()

In [50]:
# Set User 1 to the smaller id and user2 to the larger id.... then remove duplicates
edgesRDD = edgesWithDuplicates.select("userId", "userId2", "count").rdd.map(lambda x: (min(x["userId"], x["userId2"]), max(x["userId"], x["userId2"]), x["count"]))

In [51]:
edges1 = spark.createDataFrame(edgesRDD, ["src", "dst", "count"])
edges = edges1.dropDuplicates()
edges.count()

In [52]:
edges.agg({"count": 'avg'}).show()

In [53]:
# filtering the edges which are greater than average
edgesFiltered = edges.filter("count>15")
edgesFiltered.count()

#### Vertices

In [55]:
verticesList = edgesFiltered.select("src").distinct().toPandas().values.tolist() + edgesFiltered.select("dst").distinct().toPandas().values.tolist()
verticesFiltered = spark.createDataFrame(verticesList, ["id"]).distinct()
verticesFiltered.show(5)

In [56]:
verticesFiltered.count()

#### Graph

In [58]:
# graph
graph = GraphFrame(verticesFiltered, edgesFiltered)
display(graph.edges)

# TASK 5

In [60]:
# checkpoint
sc.setCheckpointDir("dbfs:/tmp/group1/checkpoint")
# Computes the connected components of the graph.
connectedComp = graph.connectedComponents()  # DataFrame with new vertices column “component”
connectedComp.count()
display(connectedComp.sort("component", ascending = False))

#### Subgraph

In [62]:
# finding max component for subgraph using connectedComp

max_component = connectedComp.filter(connectedComp["component"]==3)
edges_max_component_src = edges.join(max_component,max_component.id == 
                          edgesFiltered.src,'inner').select(edgesFiltered.src,edgesFiltered.dst)
edges_max_component = edges_max_component_src.join(max_component,max_component.id == 
                          edges_max_component_src.dst,'inner').select(edgesFiltered.src,edgesFiltered.dst)
display(edges_max_component)

In [63]:
edges_max_component.write.mode('overwrite').parquet("edges_max_component.parquet")

In [64]:
test = spark.read.parquet("edges_max_component.parquet")
display(test)

# TASK 6

In [66]:
# set the number of top_k
top_k = 3

#### Sample graph for testing GN implementation.

In [68]:
vertices = sqlContext.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 36),
  ("g", "Gabby", 60)], ["id", "name", "age"])

In [69]:
edges = sqlContext.createDataFrame([
  ("a", "b", "friend"),
  ("b", "a", "friend"),
  ("a", "c", "friend"),  
  ("c", "a", "friend"),
  ("b", "c", "friend"),
  ("c", "b", "friend"),
  ("b", "d", "friend"),
  ("d", "b", "friend"),
  ("d", "e", "friend"),
  ("e", "d", "friend"),
  ("d", "g", "friend"),
  ("g", "d", "friend"),
  ("e", "f", "friend"),
  ("f", "e", "friend"),
  ("g", "f", "friend"),
  ("f", "g", "friend"),
  ("d", "f", "friend"),
  ("f", "d", "friend")
], ["src", "dst", "relationship"])

In [70]:
# create metworkx graph
src = edges.toPandas()["src"].tolist()
dst = edges.toPandas()["dst"].tolist()
edges_new = pd.DataFrame()
edges_new['src'] = src
edges_new['dst'] = dst
edges_new[['src','dst']]
G=nx.from_pandas_edgelist(edges_new,'src','dst')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3742974253834256> in <module> 
 1 src = edges . toPandas ( ) [ "src" ] . tolist ( ) 
 2 dst = edges . toPandas ( ) [ "dst" ] . tolist ( ) 
 ----> 3 edges_new = pd . DataFrame ( ) 
 4 edges_new [ 'src' ] = src
 5 edges_new [ 'dst' ] = dst

 NameError : name 'pd' is not defined

In [71]:
# list all the nodes of the graph
G.number_of_nodes()
G.nodes

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3742974253834259> in <module> 
 1 # list all the nodes of the graph 
 ----> 2 G . number_of_nodes ( ) 
 3 G . nodes

 NameError : name 'G' is not defined

In [72]:
G_1 = sc.parallelize(edges.collect())

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4427340531185765> in <module> 
 ----> 1 G_1 = sc . parallelize ( G . collect ( ) ) 

 NameError : name 'G' is not defined

In [73]:
G_2 = G_1.map(lambda x : (x[0],x[1])).collect()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4427340531185766> in <module> 
 ----> 1 G_2 = G_1 . map ( lambda x : ( x [ 0 ] , x [ 1 ] ) ) . collect ( ) 

 NameError : name 'G_1' is not defined

In [74]:
def SSSP(G,s):
    S = [] # S is a dictionary. It stores the visited nodes
    P = {} # A dictionary to store parent node
    for v in G:
        P[v] = []
    sigma = dict.fromkeys(G, 0.0)    # sigma[v]=0 for v in G
                                    # set label for each node with 0 
    D = {}  #store the shortest path for each node to the begin node 
    sigma[s] = 1.0 # starting from vertex s and set the label of 1
    D[s] = 0 #initialise all the node to the begin node is 0
    Q = [s] # stack
    while Q:   # use BFS to find shortest paths
        v = Q.pop(0)
        S.append(v)
        Dv = D[v]
        sigmav = sigma[v]
        for w in G[v]:
            if w not in D:
                Q.append(w)
                D[w] = Dv + 1
#                 print("D[w] = ",w,D[w])
            if D[w] == Dv + 1:   # this is a shortest path, count paths
                sigma[w] += sigmav
#                 print("sigma[w] = ",w,sigma[w])
                P[w].append(v)  # predecessors
#     print (S)
#     print(P)
#     print(sigma)
    return S, P, sigma 

In [75]:
def betweenness(G):
    betweenness = dict.fromkeys(G, 0.0)  # b[v]=0 for v in G
    # b[e]=0 for e in G.edges()
    betweenness.update(dict.fromkeys(G.edges(), 0.0))
    for s in G:
        # single source shortest paths
        # use BFS
        S, P, sigma = SSSP(G,s) # step 1 from lecture note
        # accumulation
        betweenness = step_2_edges(betweenness, S, P, sigma, s) # step 2 and 3 from lecture note
    # rescaling
    for n in G:  # remove nodes to only return edges
        del betweenness[n]
    betweenness = rescale_e(betweenness, len(G))
    return betweenness

In [76]:
#step 2
def step_2_edges(betweenness, S, P, sigma, s):
    delta = dict.fromkeys(S, 0)
    while S:
        w = S.pop() # Poping value from last
        ce = (1 + delta[w]) / sigma[w] # calculating coefficient value by using weight of edge
        for v in P[w]:
            c = sigma[v] * ce
            # set order of the node from bottom to top
            if (v, w) not in betweenness:
                betweenness[(w, v)] += c
            else:
                betweenness[(v, w)] += c
            delta[v] += c
        if w != s:
            betweenness[w] += delta[w]
    return betweenness

In [77]:
# rescaling
# Finally, the true betweenness is obtained by dividing the result by two, since every shortest path will be discovered twice, once for each of its endpoints.
def rescale_e(betweenness, n):
    scale = 0.5
    for v in betweenness:
         betweenness[v] *= scale
    return betweenness

File "<command-4427340531185222>" , line 5 
 for v in betweenness: 
 ^
 IndentationError : unexpected indent

In [78]:
# compute the edge betweenness
def CmtyGirvanNewmanStep(G):
    init_ncomp = nx.number_connected_components(G)    #number of components
    ncomp = init_ncomp
    while ncomp <= init_ncomp:
        #return a dict betweenness as the value
#         bw = nx.edge_betweenness(G, weight='weight')    #edge betweenness for G 
        bw = betweenness(G)
        print("current betweenness of each edge")
        print(bw)
        #find the edge with max centrality
        max_ = max(bw.values())
        #find the edge with the highest centrality and remove all of them if there is more than one!
        for k, v in bw.items():
            if float(v) == max_:
                G.remove_edge(k[0],k[1])    #remove the central edge
        ncomp = nx.number_connected_components(G)    #recalculate the no of components

In [79]:
# This method runs GirvanNewman algorithm and list communities as output after removing the top-K edges
def runGirvanNewman(G,k):
    while k>0:    
        CmtyGirvanNewmanStep(G)
        comps = list(nx.connected_components(G)) 
        print("communities:")
        print(comps)
        k = k-1
        if G.number_of_edges() == 0:
            break

In [80]:
def main():
    runGirvanNewman(G,top_k)
if __name__ == "__main__":
      main()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-967392378204656> in <module> 
 2 runGirvanNewman ( G , top_k ) 
 3 if __name__ == "__main__" : 
 ----> 4 main ( ) 

 <command-967392378204656> in main () 
 1 def main ( ) : 
 ----> 2 runGirvanNewman ( G , top_k ) 
 3 if __name__ == "__main__" : 
 4 main ( ) 

 <command-2279729652484599> in runGirvanNewman (G, k) 
 2 def runGirvanNewman ( G , k ) : 
 3 while k > 0 : 
 ----> 4 CmtyGirvanNewmanStep ( G ) 
 5 comps = list ( nx . connected_components ( G ) ) 
 6 print ( "communities:" ) 

 <command-2279729652484598> in CmtyGirvanNewmanStep (G) 
 6 #return a dict betweenness as the value 
 7 # bw = nx.edge_betweenness(G, weight='weight') #edge betweenness for G 
 ----> 8 bw = betweenness ( G ) 
 9 print ( "current betweenness of each edge" ) 
 10 print ( bw ) 

 <command-4427340531185220> in betweenness (G) 
 12 for n in G : # remove nodes to only return edges 
 13 del betweenness [ n ] 
 ---> 14 betweenness = _rescale_e ( betweenness , len ( G ) ) 
 15 return betweenness

 NameError : name '_rescale_e' is not defined

##Girvan-Newman algorithm for subgraph

In [82]:
#create networkx graph
subG = spark.read.parquet("edges_max_component.parquet")
subG_1 = sc.parallelize(subG.collect())
subG_2 = subG_1.map(lambda x : (x[0],x[1])).collect()

Error while obtaining a new communication channel

In [83]:
sub_src = subG_2.toPandas()["src"].tolist()
sub_dst = subG_2.toPandas()["dst"].tolist()
edges_sub = pd.DataFrame()
edges_sub['src'] = sub_src
edges_sub['dst'] = sub_dst
edges_sub[['src','dst']]
sub_G=nx.from_pandas_edgelist(edges_sub,'src','dst')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4427340531185760> in <module> 
 ----> 1 sub_src = subG . toPandas ( ) [ "src" ] . tolist ( ) 
 2 sub_dst = subG . toPandas ( ) [ "dst" ] . tolist ( ) 
 3 edges_sub = pd . DataFrame ( ) 
 4 edges_sub [ 'src' ] = sub_src
 5 edges_sub [ 'dst' ] = sub_dst

 NameError : name 'subG' is not defined

In [84]:
runGirvanNewman(sub_G,top_k)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4427340531185226> in <module> 
 ----> 1 runGirvanNewman ( sub_G , top_k ) 

 NameError : name 'runGirvanNewman' is not defined